In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy import read
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from obspy.signal.filter import highpass

for csv_file in csv_files:
    cat_directory = './data/mars/test/data'
    cat_file = cat_directory + '/' + csv_file
    cat = pd.read_csv(cat_file)
    csv_file = f'{cat_file}'
    mseed_file = csv_file.replace('csv', 'mseed')
    st = read(mseed_file)
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data
    starttime = tr.stats.starttime.datetime
    arrival = (arrival_time - starttime).total_seconds()

    corner_freq = 1.0
    tr_filtered = tr.copy()
    tr_filtered.data = highpass(tr.data, corner_freq, df=tr.stats.sampling_rate)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
    ax1.plot(tr_times, tr.data, label='Signal original')
    ax1.set_ylabel('Amplitude')
    ax1.legend()
    ax1.set_title('Signal original')
    ax2.plot(tr_times, tr_filtered.data, label='Signal filtré', color='orange')
    ax2.set_xlabel('Temps (s)')
    ax2.set_ylabel('Amplitude')
    ax2.legend()
    ax2.set_title(f'Signal filtré (passe-haut {corner_freq} Hz)')
    plt.tight_layout()
    plt.show()

    df = tr.stats.sampling_rate
    sta_len = 120
    lta_len = 600

    cft = classic_sta_lta(tr_filtered.data, int(sta_len * df), int(lta_len * df))

    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    ax.plot(tr_times, cft)
    ax.set_xlim([min(tr_times), max(tr_times)])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Characteristic function')
    ax.set_title(f'{mseed_file}', fontweight='bold')

    thr_on = 4
    thr_off = 1.5
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))

    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    ax.plot(tr_times, tr_filtered.data, label='Sismogramme')

    event_times = []
    amplitudes = []
    on_label_used = False
    off_label_used = False
    for i in range(len(on_off)):
        triggers = on_off[i]
        event_start = tr_times[triggers[0]]
        event_end = tr_times[triggers[1]]
        ax.axvline(x=event_start, color='red', label='Début événement' if not on_label_used else '')
        ax.axvline(x=event_end, color='purple', label='Fin événement' if not off_label_used else '')
        event_times.append((event_start, event_end))
        amplitudes.append(np.max(tr_filtered.data[triggers[0]:triggers[1]]))
        on_label_used = True
        off_label_used = True

    ax.set_xlim([min(tr_times), max(tr_times)])
    ax.set_xlabel('Temps (s)')
    ax.set_ylabel('Amplitude')
    ax.legend()
    plt.title('Sismogramme avec détections STA/LTA')
    plt.show()

    signal_power = np.mean(tr_filtered.data ** 2)
    noise_power = np.mean((tr_filtered.data[triggers[1]:] - np.mean(tr_filtered.data[triggers[1]:])) ** 2)
    snr = signal_power / noise_power

    summary_df = pd.DataFrame({
        'Event Start (s)': [event[0] for event in event_times],
        'Event End (s)': [event[1] for event in event_times],
        'Amplitude': amplitudes,
        'SNR': [snr] * len(event_times)
    })

    summary_file = f'{mseed_file}_summary.csv'
    summary_df.to_csv(summary_file, index=False)

    print(f"Résumé sauvegardé dans : {summary_file}")

FileNotFoundError: [Errno 2] No such file or directory: './data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.mseed_summary.mseed'